### 목표

0. 특정 상품의 정보와 이미지 얻기
1. traversing 


### hierarchy

* department -> women, men, boy, girl..
* women -> clothing, shoes,... 등등 (카테고리)
* clothing -> dresses, tops,tees & blouses, sweaters, Fashion hoodles & sweatShirts..등등

__이중에 women - cloting - sweaters 를 예로 들어보면__

1. 1페이지 주소
https://www.amazon.com/s/ref=sr_pg_1?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&bbn=16225018011&ie=UTF8&qid=1545225556

2. 2페이지 주소
https://www.amazon.com/s/ref=sr_pg_2?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&page=2&bbn=16225018011&ie=UTF8&qid=1545225398

* 페이지 변환 (1, 2, 3..) ( __page = k __)
    url에서, 처음에 있는 "pg_1?" 부분 말고 뒤에 있는 "page = k"  에서 k를 바꾸며 조정

* 특정 페이지의, 모든 품목 살피기 (__id = "result_k"__) 
    ul id = "s-results-list-arf"  
        <li id = "result_k" ..  모든 품목을 k를 증가시키며 서치, 없으면 다음페이지로 넘어감. __단, k는 누적해서 증가__
                <div class = "a-row a-spacing-none sx-line-clamp-4">
                    <a class ="a-link-normal s-access-detail-page ...   이 a tag의 href 속성으로 링크 얻을 수 있음..

* 특정 품목의 정보 크롤링 (__id = "feature-bullets"__)
* 특정 품목의 이미지 (__div id = "imgTagWeapperid" 안의 img 태그의 src__) 

대충 아래 처럼 생김

```
<div id="imgTagWrapperId" class="imgTagWrapper" style="height: 565px;">
	            <img alt="LARACE Women 3/4 Sleeve Tunic Top Loose Fit Flare T-Shirt(L, Black)" src="https://images-na.ssl-images-amazon.com/images/I/51aa-w%2BPlOL._UY741_.jpg" data-old-hires="https://images-na.ssl-images-amazon.com/images/I/51aa-w%2BPlOL._UL1024_.jpg" class="a-dynamic-image  a-stretch-vertical" id="landingImage" data-a-dynamic-image="{&quot;https://images-na.ssl-images-amazon.com/images/I/51aa-w%2BPlOL._UY741_.jpg&quot;:[741,526],&quot;https://images-na.ssl-images-amazon.com/images/I/51aa-w%2BPlOL._UY879_.jpg&quot;:[879,624],&quot;https://images-na.ssl-images-amazon.com/images/I/51aa-w%2BPlOL._UY445_.jpg&quot;:[445,316],&quot;https://images-na.ssl-images-amazon.com/images/I/51aa-w%2BPlOL._UY500_.jpg&quot;:[500,355],&quot;https://images-na.ssl-images-amazon.com/images/I/51aa-w%2BPlOL._UY606_.jpg&quot;:[606,430],&quot;https://images-na.ssl-images-amazon.com/images/I/51aa-w%2BPlOL._UY679_.jpg&quot;:[679,482],&quot;https://images-na.ssl-images-amazon.com/images/I/51aa-w%2BPlOL._UY550_.jpg&quot;:[550,390]}" style="max-width: 401.066px; max-height: 565px;">
	        </div>
```

__알고리즘 대충 이렇게..__

1. 시작 url은 크롤링 하고싶은 카테고리의 url
2. url의 "page = k" 로 페이지를 바꿔감 (loop 1)
3. 각 페이지당 li id = "result_k" 엘리먼트 에 있는 a 태그로 각 품목으로 이동하는 링크를 얻음.. (loop2) 

### 1. 상품 하나 크롤링 : features, img

In [16]:
import os
print (os.getcwd())

doubleDic = {1 : 100, 2 : 200, 3 : {30:300, 40:400, 50:500}} 
print(doubleDic)
print(len(doubleDic[3]))

/home/user/ML/2_2project
{1: 100, 2: 200, 3: {30: 300, 40: 400, 50: 500}}
3


In [13]:
from urllib.request import Request, urlopen #강사님이 꾸리다고 절대 쓰지 말라하심! requests로 바꾸자.. 
#requrests는 그냥 써도 되는데, urllib는 브라우저 headr추가하지 않으면 아마존이 막아버림
from bs4 import BeautifulSoup

imgPath = "/home/user/ML/2_2project/imgContainer/"# + k.png
featurePath = "/home/user/ML/2_2project/featureContainer/"# + .txt

def getFeature(url, k): # k : 몇번째 품목인지 (result_k)
    req = Request(url)
    req.add_header('User-agent', 'Mozilla/5.0') #브라우저 인척
    f = urlopen(req)
    soup = BeautifulSoup(f, "lxml") #아마존은 html.parser가 안되나 봅니다.
    allSpan = soup.find(id = "feature-bullets").find_all("span")
    features = []
    for span in allSpan:
        features.append(span.text.strip())
        
    #feature 저장
    with open(featurePath + str(k) + ".txt", mode = 'wt', encoding = 'utf-8') as featureSaveFile:
        for feature in features:
            featureSaveFile.write(feature)
            #featureSaveFile.write('\n'.join(feature))
    
    #이미지 저장
    imgsrc = soup.find(id = "imgTagWrapperId").find("img")['src']
    req2 = Request(imgsrc)
    req2.add_header('User-agent', 'Mozilla/5.0')
    f2 = urlopen(req2).read()
    imgSaveFile = open(imgPath + str(k) + ".jpg", 'wb')
    imgSaveFile.write(f2)
    imgSaveFile.close()
    print(features) #프린트 말고 저장하자~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
getFeature(url, 1)

#savePath = "htmlContainer/amazon.html"
#with open(savePath, 'wb') as saveFile:  
#   saveFile.write(html)
#print('html 저장 완료')

['95% Rayon, 5% Spandex', 'Feature 3/4 Sleeve, round neck, flare hem, casual style, flowy, lightweight and comfy fabric.', 'Designed cutting to make you look slim, and long enough to hide belly and hips.', 'Easy to pair with jeans, shorts, skinny leggings for the decent look. Great for leggings with scarf, and boots in fall and winter.', 'Super soft, stretchy and lightweight, can be easily dress up or dress down. Occasion: Dating, Party, Evening, Casual, Travel, Home, Office, Vocation, Beach, maternity etc.', 'Wash Instruction: Hand wash recommended; Machine wash cold / Do Not Bleach / Hang or Line Dry / Please refer to the washing lable inside.']


### 2. traversing 적용 (1차 완성 코드)

In [25]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

#여성-스웨터
#https://www.amazon.com/s/ref=sr_pg_1?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&page=1&bbn=16225018011&ie=UTF8&qid=1545359281
#pre = "https://www.amazon.com/s/ref=sr_pg_1?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&page="
#pageNum = 1 #몇페이지 인지
#post = "&bbn=16225018011&ie=UTF8&qid=1545359281"

#여성-드레스
#https://www.amazon.com/s/ref=sr_pg_2?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1045024&page=1&bbn=16225018011&ie=UTF8&qid=1545371752
pre = "https://www.amazon.com/s/ref=sr_pg_2?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1045024&page="
post = "&bbn=16225018011&ie=UTF8&qid=1545371752"

imgPath = "imgContainer/"# + k.png
featurePath = "featureContainer/"# + k.txt

#특정 상품 하나의 페이지 크롤링 하는 함수
def getFeature(url, k): # k : 몇번째 품목인지 (result_k)
    req = Request(url)
    req.add_header('User-agent', 'Mozilla/5.0')
    f = urlopen(req)
    soup = BeautifulSoup(f, "lxml") 
    allSpan = soup.find(id = "feature-bullets").find_all("span")
    features = [] 
    for span in allSpan:
        features.append(span.text.strip())
    #feature 저장
    with open(featurePath + str(k) + ".txt", mode = 'wt', encoding = 'utf-8') as featureSaveFile:
        for feature in features:
            featureSaveFile.write(feature)
    #이미지 저장
    imgsrc = soup.find(id = "imgTagWrapperId").find("img")['src']
    req2 = Request(imgsrc)
    req2.add_header('User-agent', 'Mozilla/5.0')
    f2 = urlopen(req2).read()
    imgSaveFile = open(imgPath + str(k) + ".jpg", 'wb')
    imgSaveFile.write(f2)
    imgSaveFile.close()
    #print(features)
    
maxPage = 1 #몇페이지까지 크롤링 할지..  
itemNum = 0 #상품 인덱스, 1번부터 시작 max
for pageNum in range(1, maxPage + 1):
    print(" << ", pageNum, "페이지 뒤지기>>")
    url = pre + str(pageNum) + post 
    req = Request(url)
    req.add_header('User-agent', 'Mozilla/5.0')
    f = urlopen(req)
    soup = BeautifulSoup(f, "lxml") #아마존은 html.parser 가 안되나 봅니다 .. 
    
    while True: #너무 오래 걸리니까 일단 지우자
    #for i in range(5) :
        item = soup.find(id = "result_" + str(itemNum))  #못찾을때까지 반복
        if( item == None ) :
            print(itemNum, "번째 상품은 이 페이지에 없다. 다음페이지로")
            break;
        else:
            #itemURL = item.find("a")['href']  # ==> bestSeller인 애들은 다른 a태그가 하나 더 먼저 있어서 이렇게 하면 안됨
            itemURL_list = item.find_all("a")
            itemURL = itemURL_list[0]["href"] #---------------좋은 방법이 아닌 것 같아 멍청아..----------------------------------------------------------
            if(itemURL_list[0]['href'][0:3] == "/gp"):
                itemURL = itemURL_list[1]['href']
            #--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
            print(itemNum, "번 상품의 주소 :  ", itemURL)
            getFeature(itemURL, itemNum)  #itemURL을 통하여, feature(text)와 상품 이미지를 저장.
            itemNum += 1
        

 <<  1 페이지 뒤지기>>
0 번 상품의 주소 :   https://www.amazon.com/DEARCASE-Womens-Sleeve-Casual-T-Shirt/dp/B01AA6PPXU/ref=sr_1_1/137-3832329-2212467?s=fashion-womens-intl-ship&ie=UTF8&qid=1545371874&sr=1-1
1 번 상품의 주소 :   https://www.amazon.com/Unbranded-Womens-Sleeve-Pocket-T-Shirt/dp/B0754WPYHT/ref=sr_1_2/137-3832329-2212467?s=fashion-womens-intl-ship&ie=UTF8&qid=1545371874&sr=1-2
2 번 상품의 주소 :   https://www.amazon.com/LaSuiveur-Womens-Christmas-Pullover-Flared/dp/B01MD0HJDG/ref=sr_1_3/137-3832329-2212467?s=fashion-womens-intl-ship&ie=UTF8&qid=1545371874&sr=1-3
3 번 상품의 주소 :   https://www.amazon.com/MissMay-Womens-Vintage-Cocktail-C-black_0/dp/B01MRPYLEI/ref=sr_1_4/137-3832329-2212467?s=fashion-womens-intl-ship&ie=UTF8&qid=1545371874&sr=1-4
4 번 상품의 주소 :   https://www.amazon.com/AUSELILY-Womens-Sleeve-Pleated-Pockets/dp/B075ZPM2YD/ref=sr_1_5/137-3832329-2212467?s=fashion-womens-intl-ship&ie=UTF8&qid=1545371874&sr=1-5


# (문제) 밴 

500장 정도 모으다가 블락 당했다... 시간이 지나면 풀리는데 또 하면 또 막힌다.. 

__참고__ : 

1. 천천히 크롤링 (랜덤한 시간 간격 부여)
    => 또 밴당함
2. 아이피 우회 (Vpn, proxy, Tor) 
    => 나중에 아예 IP 차단 당하면 그때 쓰자
3. 셀레니움
   *  => 직접 브라우저를 제어? 웹앱 테스트로 쓰이는 모듈. 
   *  => __참고__ : 
   *  => 단점: 느림.. 도덕.. 

### 3. 셀레니움으로 바꾼 코드

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request as ur
import random
import time

driver = webdriver.Chrome("/home/user/ML/2_2project/chromedriver") 
driver.implicitly_wait(5) # 암묵적으로 웹 자원을 (최대) 5초 기다리기

#여성-스웨터
#https://www.amazon.com/s/ref=sr_pg_1?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&page=1&bbn=16225018011&ie=UTF8&qid=1545359281
pre = "https://www.amazon.com/s/ref=sr_pg_1?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&page="
#pageNum = 1 #몇페이지 인지
post = "&bbn=16225018011&ie=UTF8&qid=1545359281"

imgPath = "imgContainer/sweater"# + k.png
featurePath = "featureContainer/sweater"# + k.txt

#특정 상품 하나의 페이지 크롤링 하는 함수
def getFeature(url, k): # k : 몇번째 품목인지 (result_k)
    driver.get(url)
    #time.sleep(random.randint(1, 5))
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    allSpan = soup.find(id = "feature-bullets").find_all("span")
    features = [] 
    for span in allSpan:
        features.append(span.text.strip())
    #feature 저장
    with open(featurePath + str(k) + ".txt", mode = 'wt', encoding = 'utf-8') as featureSaveFile:
        for feature in features:
            featureSaveFile.write(feature)
    #이미지 저장
    imgsrc = soup.find(id = "imgTagWrapperId").find("img")['src']
    ur.urlretrieve(imgsrc, imgPath + str(k) + ".jpg") #url, 경로
    
maxPage = 200 #몇페이지까지 크롤링 할지..  
itemNum = 4747 #상품 인덱스, 1번부터 시작 max
for pageNum in range(198, maxPage + 1) :
    #if ( pageNum % 10 == 0 and pageNum != 120) :
        #time.sleep(600)
    print(" << ", pageNum, "페이지 뒤지기>>")
    url = pre + str(pageNum) + post 
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    
    while True: #너무 오래 걸리니까 일단 지우자
        item = soup.find(id = "result_" + str(itemNum))  #못찾을때까지 반복
        if( item == None ) :
            print(itemNum, "번째 상품은 이 페이지에 없다. 다음페이지로")
            break;
        else:
            #itemURL = item.find("a")['href']  # ==> bestSeller인 애들은 다른 a태그가 하나 더 먼저 있어서 이렇게 하면 안됨
            itemURL_list = item.find_all("a")
            itemURL = itemURL_list[0]["href"] #---------------좋은 방법이 아닌 것 같아 멍청아..----------------------------------------------------------
            if(itemURL_list[0]['href'][0:3] == "/gp"):
                itemURL = itemURL_list[1]['href']
            #--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
            print(itemNum, "번 상품의 주소 :  ", itemURL)
            getFeature(itemURL, itemNum)  #itemURL을 통하여, feature(text)와 상품 이미지를 저장.
            itemNum += 1
        

 <<  198 페이지 뒤지기>>
4747 번 상품의 주소 :   https://www.amazon.com/Womens-Sleeve-Cardigan-Pockets-XX-Large/dp/B07CDDRFLK/ref=sr_1_4748/137-0897089-1848847?s=fashion-womens-intl-ship&ie=UTF8&qid=1545844802&sr=1-4748
4748 번 상품의 주소 :   https://www.amazon.com/Dorimis-Turtle-Sweater-Pullover-Sweatshirt/dp/B07K7JVQ4N/ref=sr_1_4749/137-0897089-1848847?s=fashion-womens-intl-ship&ie=UTF8&qid=1545844802&sr=1-4749
4749 번 상품의 주소 :   https://www.amazon.com/GEESENSS-Womens-Sweater-Outerwear-Cardigan/dp/B076Q6F5Y2/ref=sr_1_4750/137-0897089-1848847?s=fashion-womens-intl-ship&ie=UTF8&qid=1545844802&sr=1-4750
4750 번 상품의 주소 :   https://www.amazon.com/PJ-Salvage-Womens-Sherpa-Charcoal/dp/B0758M6352/ref=sr_1_4751/137-0897089-1848847?s=fashion-womens-intl-ship&ie=UTF8&qid=1545844802&sr=1-4751
4751 번 상품의 주소 :   https://www.amazon.com/Royal-Robbins-Summertime-Pointelle-Cardigan/dp/B00ZW6VTOS/ref=sr_1_4752/137-0897089-1848847?s=fashion-womens-intl-ship&ie=UTF8&qid=1545844802&sr=1-4752
4752 번째 상품은 이 페이지에 없다. 다음페이지로
 <

# AWS 클라우드 가입부터 활용까지.. 

## (문제)

1. item index를 하나씩 건너뛰는가 경우 가끔 존재
    * 예) result_100 에서 result_102로 넘어간다던가.. 
    * => 못찾은 경우, 바로 페이지 전환을 하지 않고, 누락된 것인지 확인하는 코드 필요 
2. html 내용이 조금 다른 상품이 있음
    * 예)  https://www.amazon.com/Pinnacle-Mercantile-Drawstring-Heavy-Duty-Puncture-Resistant/dp/B07GDM7KHQ/ref=sr_1_3066?s=fashion-womens-intl-ship&ie=UTF8&qid=1545582401&sr=1-3066
    * #feature-bullets > span 이없고, 아래와 같음.. 
    * __#feature-bullets-btf > table > tbody > tr > td > div > ul > li:nth-child(1)__
    * 근데 얘만 그런 것 같아서.. 일단 코드를 수정하지 않기로!
3. 특정 일을 기준으로 상품번호가 많이 밀렸음
    * 원래 118페이지에 3066 번째 상품이 있었는데, 128페이지로 밀려있음.. 
4. feature 없는 옷도 있음 
    * => feature-bullets id는 있는데 비어있음
    * 예) https://www.amazon.com/Tommy-Hilfiger-Sweater-Polka-dot-X-Large/dp/B01N3NG5C2/ref=sr_1_5550?s=fashion-womens-intl-ship&ie=UTF8&qid=1545903866&sr=1-5550
    * 예) https://www.amazon.com/Jumper-Knitted-Pullover-Sweaters-Sweater/dp/B07J9XL3NJ/ref=sr_1_5551?s=fashion-womens-intl-ship&ie=UTF8&qid=1545903866&sr=1-5551


__누락) 3134, 3231, 4096 ~ 4100, 4746__

### 4. getFeature 메소드 수정 + 예외처리

1. __json 형식으로 저장__
    * { "id" : "B075DDFCC", "title" : " ... ", "category" : "스웨터", "features" : {1 : "span1내용" , 2 : "span2 내용", ...}  } __리스트? 객체?__ 
    * 1. feature 하나당 하나의 객체로. 
    * 2. 상품의 id와 title, category( 스웨터 인지 뭔지..) 까지 추가 

2. 좀더 큰 이미지로 저장
    * landingImage - hires
    
3. 저장명은 id로 !
    * gp 뒤에있는 코드 

4. 추가로 해야할일 
    * related .... 로 긁어보기..
    * tfidf 
    * LDA 토픽분석


url = "https://www.amazon.com/dp/B074YKXPC8"

In [68]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request as ur
import random
import time
import json

driver = webdriver.Chrome("/home/user/ML/2_2project/chromedriver") 

#여성-스웨터
#https://www.amazon.com/s/ref=sr_pg_1?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&page=1&bbn=16225018011&ie=UTF8&qid=1545359281
pre = "https://www.amazon.com/s/ref=sr_pg_1?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&page="
#pageNum = 1 #몇페이지 인지
post = "&bbn=16225018011&ie=UTF8&qid=1545359281"

imgPath = "imgContainer/sweater"# + k.png
featurePath = "featureContainer/sweater"# + k.txt
category = "sweater"

#특정 상품 하나의 페이지 크롤링 하는 함수
def getFeature(url, k): # k : 몇번째 품목인지 (result_k)
    info = {}  #저장할 json 파일
    id = url.split('/dp/')[1].split('/')[0] # 상품의 id 얻기                       __ id.jpg, id.json  
    title = url.split('/dp/')[0].split('/')[3] # 상품의 title 얻기
    driver.get(url)
    #time.sleep(random.randint(1, 5))
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    allSpan = soup.find(id = "feature-bullets").find_all("span")
    features = []
    for span in allSpan:
        no_blank_string = ' '.join(span.text.split())
        features.append(no_blank_string)
    #이미지 저장
    #imgsrc = soup.find(id = "imgTagWrapperId").find("img")['src'] 
    imgURL = soup.find(id = 'landingImage')['data-old-hires'] #더 큰 파일
    ur.urlretrieve(imgURL, "imgContainer/" + id + ".jpg") #url, 경로
    # category 모두 긁기..
    allLi = soup.find(id = "wayfinding-breadcrumbs_feature_div")#.findAll("li")
    if(allLi == None) : #카테고리 없는 애들도 있다
        #print("empty")
        attStr = "empty"
    else:
        allLi = allLi.findAll("li")
        allStr = ""
        for li in allLi :
            a = li.find('a')
            if( a == None ):
                allStr += " > "
            else:
                allStr += a.text.strip()
    info = {
        'id' : id,
        'title' : title,
        'category' : category,
        'fullCategory' : allStr,
        'features' : features
    }
    #print(features)
    print(info)
    json.dump(info, open(f'featureContainer/' + id + '.json', 'w+'))
    
url = "https://www.amazon.com/dp/B01AA6PPXU" 
getFeature(url, 1)

IndexError: list index out of range

### 5. 이제 돌리면 된다..

In [69]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request as ur
import random
import time
import json

driver = webdriver.Chrome("/home/user/ML/2_2project/chromedriver") 

#여성-스웨터
#https://www.amazon.com/s/ref=sr_pg_1?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&page=1&bbn=16225018011&ie=UTF8&qid=1545359281
pre = "https://www.amazon.com/s/ref=sr_pg_1?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&page="
#pageNum = 1 #몇페이지 인지
post = "&bbn=16225018011&ie=UTF8&qid=1545359281"

imgPath = "imgContainer/sweater"# + k.png
featurePath = "featureContainer/sweater"# + k.txt
category = "sweater"

#특정 상품 하나의 페이지 크롤링 하는 함수
def getFeature(url, k): # k : 몇번째 품목인지 (result_k)
    info = {}  #저장할 json 파일
    id = url.split('/dp/')[1].split('/')[0] # 상품의 id 얻기                       __ id.jpg, id.json  
    title = url.split('/dp/')[0].split('/')[3] # 상품의 title 얻기
    driver.get(url)
    #time.sleep(random.randint(1, 5))
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    allSpan = soup.find(id = "feature-bullets").find_all("span")
    features = []
    for span in allSpan:
        no_blank_string = ' '.join(span.text.split())
        features.append(no_blank_string)
    #이미지 저장
    #imgsrc = soup.find(id = "imgTagWrapperId").find("img")['src'] 
    imgURL = soup.find(id = 'landingImage')['data-old-hires'] #더 큰 파일
    ur.urlretrieve(imgURL, "imgContainer/" + id + ".jpg") #url, 경로
    # category 모두 긁기..
    allLi = soup.find(id = "wayfinding-breadcrumbs_feature_div")#.findAll("li")
    if(allLi == None) : #카테고리 없는 애들도 있다
        allStr = ""
    else:
        allLi = allLi.findAll("li")
        allStr = ""
        for li in allLi :
            a = li.find('a')
            if( a == None ):
                allStr += " > "
            else:
                allStr += a.text.strip()
    info = {
        'id' : id,
        'title' : title,
        'category' : category,
        'fullCategory' : allStr,
        'features' : features
    }
    #print(features)
    print(info)
    json.dump(info, open(f'featureContainer/' + id + '.json', 'w+'))
    
maxPage = 100 #몇페이지까지 크롤링 할지..  
itemNum = 591 #상품 인덱스, 1번부터 시작 max
for pageNum in range(25, maxPage + 1):
    print(" << ", pageNum, "페이지 뒤지기 >>")
    url = pre + str(pageNum) + post 
    print(url)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    
    while True: 
        item = soup.find(id = "result_" + str(itemNum))  #못찾을때까지 반복
        if( item == None ):
            if (soup.find(id = "result_" + str(itemNum + 1)) != None ): #혹시 하나가 누락 된 걸수도 있음! 
                itemNum += 1
                continue;
            else: 
                print(itemNum, "번째 상품은 이 페이지에 없다. 다음페이지로") #이럼 진짜 없는 것 .. 
                break;
        else:
            #itemURL = item.find("a")['href']  # ==> bestSeller인 애들은 다른 a태그가 하나 더 먼저 있어서 이렇게 하면 안됨
            itemURL_list = item.find_all("a")
            itemURL = itemURL_list[0]["href"] #---------------좋은 방법이 아닌 것 같아 멍청아..----------------------------------------------------------
            if(itemURL_list[0]['href'][0:3] == "/gp"):
                itemURL = itemURL_list[1]['href']
            #--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
            print(itemNum, "번 상품의 주소 :  ", itemURL)
            getFeature(itemURL, itemNum)  #itemURL을 통하여, feature(text)와 상품 이미지를 저장.
            itemNum += 1
# 590번 같이.. 동영상 포함된 경우 landingImg 에 주소가 이상해짐.. 예외처리 필요

# fullCategory : 1 : , 2: , .. 

# weakly supervised learning 

# 유니크 .. 몇개 .. 

 <<  25 페이지 뒤지기 >>
https://www.amazon.com/s/ref=sr_pg_1?fst=as%3Aoff&rh=n%3A16225018011%2Cn%3A1040660%2Cn%3A1044456&page=25&bbn=16225018011&ie=UTF8&qid=1545359281
591 번 상품의 주소 :   https://www.amazon.com/Chicwish-Lavender-Sweater-Cardigan-X-Large/dp/B078K3Z8D9/ref=sr_1_592/132-2148371-6286634?s=fashion-womens-intl-ship&ie=UTF8&qid=1546581144&sr=1-592
{'id': 'B078K3Z8D9', 'title': 'Chicwish-Lavender-Sweater-Cardigan-X-Large', 'category': 'sweater', 'fullCategory': 'Clothing, Shoes & Jewelry > Women > Clothing > Sweaters > Cardigans', 'features': ['60% Wool, 40% Polyester', 'Hand wash/ Dry clean', '- [ Style Details ] Hand knit with love by our team. Yarn balls trimmed heart shape pattern. Open front. loose fit. ribbed cuffs and hem. long sleeves.', '- [ Product Details ] Hand-knit fabric finished. Not lined. Mid-weight texture fabrication. 60% Wool, 40% Polyester. Hand wash/ Dry clean.', '[ Occasions ] Great selection for work, beach, vacation, dating and casual look, mommy & me party, c

ValueError: unknown url type: ''

### * related item 으로 탐색

#### 1. related products중 첫번째 item의 링크로 계속 이동 (X)
    * 문제) 이미 방문했던 item이 또 다시 연관 상품에 뜸 => 일정 횟수가 지나니 방문한 item들에서 빠져 나오질 못함.
    * => ex) 아래 출력 
    * 중복 인지 검사 해야함 (이미 본 상품이면 거르도록.. ) 
    * ID를 계속 저장해야 하고.. 매번 현재 ID를 누적된 ID들 에서 서치 해야하지 않나..? =>  연산노답 

In [25]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request as ur
import random
import time
import json

In [46]:
baseURL = "https://www.amazon.com/dp/"
ID = "B07G5PZ39T" #아무 ID 가지고 시작.. 

driver = webdriver.Chrome("/home/user/ML/2_2project/chromedriver") 
    
dupCounter = 0 # 중복된 item 접근수
counter = 0 # 중복 포함해서, 접근한 item 개수
categoryCounter = {} # 특정 카테고리가 몇번씩 나왔는지 

ids = [] #방문한 상품의 id를 모두 저장.. (중복 횟수 세기 위하여) 

while ( counter < 20 ) :
    driver.get(baseURL + ID)               
    html = driver.page_source              
    soup = BeautifulSoup(html, "lxml") 
    items = soup.find(id = "sp_detail").find("ol").findAll("li")
    temps = [] #특정 상품의 related item들의 id들 
    for item in items :
        id = item.find('div')['data-asin'] # relatede products의 id들만 가져오기 (url 바로 만들 수 있음!)
        temps.append(id)
    #------------------related 중에서 첫번째 item만 계속 타고간다면 --------------
    empty = []
    temps = empty.append(temps[0])
    temps = empty #--------------------------------------------------------------------------------------------
    
    for i in range(len(temps)) :
        counter += 1
        exist = False
        # -----------------------중복 검사-----------------------
        for ID in ids :
            if ( temps[i] == ID ):
                exist = True
                break
        if (exist) : #중복
            dupCounter += 1 
            print("중복")
        else : 
            print(temps[i] , " 상품 저장")
            ids.append(temps[i])
        # -------------------------------------------------------------------
        if (i == len(temps) - 1 ) :
            ID = temps[i] # 제일 마지막 item의 연관 items 들을 다시 불러와야함..
print(counter, "중에 ", dupCounter, " 개가 중복...")

B079RF7FJX  상품 저장
B079HK3WQ3  상품 저장
B076JKD2T3  상품 저장
B07H7BJ3X4  상품 저장
B01H2U1SME  상품 저장
B017OJLC74  상품 저장
B07425Y7J2  상품 저장
B07DNJD38P  상품 저장
B07DWJFNKL  상품 저장
중복
중복
중복
중복
중복
중복
중복
중복
중복
중복
중복
20 중에  11  개가 중복...


#### 2. 특정 상품 페이지에서 related products 의 링크를 여러개 저장해두고 하나씩 접근..
    * => 링크가 다 떨어지면, 가장 최근 방문 상품에서 다시 링크들을 저장함.. (목표 개수 채울 때 까지 반복..)
    * (문제) 저장하는 링크의 개수 
        * 5개 이하로 저장 : 카테고리 변화가 잘 됨, 그러나 무한중복에 빠질 가능성 높음..
        * 10개 이상 저장: 카테고리 변화가 거의 없음, 그러나 중복은 덜 해짐. (얘도 재수없으면 무한중복..)
    
#### 다른 문제들.. 

1. html구조가 다른 상품.. 
    * https://www.amazon.com/dp/B07H7NGPYW
    * related item들이 들어있는 div태그에 달린 id가 달라지는 경우가 있음 .. (sp_detail 대신 sp_detail2 라던지..) 
    * => id 의 시작이 sp_detail 인 애를 찾는 걸로 바꾸면 해결 될 것 같긴 함.. 
2. 진짜 뜬금없는 상품.. (이게 진짜 문제) 
    * https://www.amazon.com/dp/B01GU9U9KA
    * 칼.. 연관상품에 칼이 나왔는데.. 여기로 들어가니까 영원히 칼만 나온다..
    * => 카테고리가 clothing을 벗어나면 빠꾸 시키기로 해야할 것 같음..

In [55]:
def getFeature(url, k): # k : 몇번째 품목인지 (result_k)
    info = {}  #저장할 json 파일
    id = url.split('/dp/')[1].split('/')[0] # 상품의 id 얻기
    #title = url.split('/dp/')[0].split('/')[3] # 상품의 title 얻기 #아..이게 이제 안되는구나 .. 다른방법으로 얻어야해..
    title = "none"
    driver.get(url)
    #print(driver.current_url)
    #time.sleep(random.randint(1, 5))
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    # category 긁기 ..
    allLi = soup.find(id = "wayfinding-breadcrumbs_feature_div")#.findAll("li")
    if(allLi == None) : #카테고리 없는 애들도 있다
        print("no category")
    else:
        allLi = allLi.findAll("li")
        allStr = ""
        for li in allLi :
            a = li.find('a')
            if( a == None ):
                allStr += " > "
            else:
                allStr += a.text.strip()
        #print('')
        print(allStr)
    # feature 긁기 
    allSpan = soup.find(id = "feature-bullets").find_all("span")
    features = {}
    num = 1
    for span in allSpan:
        features[num] = span.text.strip() 
        num += 1
    #이미지 저장
    #imgURL = soup.find(id = 'landingImage')['data-old-hires'] #더 큰 파일
    #ur.urlretrieve(imgURL, "imgContainer/" + id + ".jpg") #url, 경로
    info = {
        'id' : id,
        'title' : title,
        'category' : category,
        'features' : features
    }
    #정보 저장
    #json.dump(info, open(f'featureContainer/' + id + '.json', 'w+'))

baseURL = "https://www.amazon.com/dp/"
ID = "B07G5PZ39T" #아무 ID 가지고 시작.. 

driver = webdriver.Chrome("/home/user/ML/2_2project/chromedriver") 
    
dupCounter = 0 # 중복된 item 접근수
counter = 0 # 중복 포함해서, 접근한 item 개수
categoryCounter = {} # 특정 카테고리가 몇번씩 나왔는지 

ids = [] #방문한 상품의 id를 모두 저장.. (중복 횟수 세기 위하여) 

while ( counter < 100 ) :
    driver.get(baseURL + ID)               
    html = driver.page_source              
    soup = BeautifulSoup(html, "lxml") 
    items = soup.find(id = "sp_detail").find("ol").findAll("li")
    temps = [] #특정 상품의 related item들의 id들 
    for item in items :
        id = item.find('div')['data-asin'] # relatede products의 id만 가져오기
        temps.append(id)
    for i in range(len(temps)) :
        counter += 1
        exist = False
        # -----------------------중복 검사-----------------------
        for ID in ids :
            if ( temps[i] == ID ):
                exist = True
                break
        if (exist) : #중복
            dupCounter += 1 
            print("중복")
        else : 
            print(temps[i] , " 상품 저장")
            ids.append(temps[i])
            getFeature(baseURL + temps[i], 1)
        # -------------------------------------------------------------------
        if (i == len(temps) - 1 ) :
            ID = temps[i] # 제일 마지막 item의 연관 items 들을 다시 불러와야함..
print(counter, "중에 ", dupCounter, " 개가 중복...")

B079RF7FJX  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters > Pullovers
B07HKP949R  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters > Pullovers
B07G5NMSG2  상품 저장
no category
B07FSQN5SC  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters > Pullovers
B078HSDVLG  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters > Pullovers
B074N4MZZQ  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters > Pullovers
B0777JZ9DM  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters > Pullovers
B074N6D71C  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters
중복
B074N5C9LZ  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters
B07G5LRVJW  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters > Pullovers
B07G5N6FSK  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters > Pullovers
B074N6JDJL  상품 저장
Clothing, Shoes & Jewelry > Women > Clothing > Sweaters
B0777K4K51  상품 저장
Clothing, Shoes & Jewelry > Women > Clo



* TF (Term Frequency) :
    * => 특정 문서에서 단어들의 빈도
    * => 특정 문서에서 특정 단어의 등장횟수 / 특정 문서의 총 단어수 

<img class="progressiveMedia-image js-progressiveMedia-image" data-src="https://cdn-images-1.medium.com/max/1600/1*HM0Vcdrx2RApOyjp_ZeW_Q.png" src="https://cdn-images-1.medium.com/max/1600/1*HM0Vcdrx2RApOyjp_ZeW_Q.png">

* IDF (Inverse Data Frequency) : 
    * => 모든 문서들을 통틀어 희귀한 단어.. 
    * log ( 총 문서 개수 / 이 단어를 포함한 문서 개수 )
    * 희귀한 단어일수록 idf 증가.
* __stopwords__ : 자주 등장하지만 중요하지 않은 단어 (다른 문서에서도 자주 등장함..) ex) the, will, you.. 

<img class="progressiveMedia-image js-progressiveMedia-image" data-src="https://cdn-images-1.medium.com/max/1600/1*A5YGwFpcTd0YTCdgoiHFUw.png" src="https://cdn-images-1.medium.com/max/1600/1*A5YGwFpcTd0YTCdgoiHFUw.png">

* TF - IDF : 
    * tf * idf 둘을 곱한 것 
    * 이 문서에서 자주 등장한 단어이면서, 다른 문서에서는 자주 등장하지 않은 단어일경우 값이 커짐.
    
<img class="progressiveMedia-image js-progressiveMedia-image" data-src="https://cdn-images-1.medium.com/max/1600/1*nSqHXwOIJ2fa_EFLTh5KYw.png" src="https://cdn-images-1.medium.com/max/1600/1*nSqHXwOIJ2fa_EFLTh5KYw.png">

<img class="graf-image" data-image-id="1*q2tRgjV_J-MLvnhwNAl0KQ.png" data-width="337" data-height="86" src="https://cdn-images-1.medium.com/max/1600/1*q2tRgjV_J-MLvnhwNAl0KQ.png">
